In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
floc = "../../input_data/historical/" #location of input data and label encoding
gcol = 'real_rce'

In [17]:
df = pd.read_csv(floc+"nexxa_combined_features.csv") #input data
df2 = pd.read_csv(floc+"nexxa_combined_targets.csv") #target data, in this case each account's actual rce


/home/brandon/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
df2.head()

service_zone utility_account_number     zone load_profile rate_class  \
0     aepcs_oh         40621000026740  PJM AEP     CCOHDM2U        NaN   
1     aepcs_oh         40621000062541  PJM AEP     CCOHRESA        820   
2     aepcs_oh         40621000065845  PJM AEP     CCOHRESA        822   
3     aepcs_oh         40621000067415  PJM AEP     CCOHRESA        822   
4     aepcs_oh         40621000106282  PJM AEP     CCOHRESA        822   

   real_icap_kw  real_nits_kw  real_rce profile  \
0         23.38         11.92     7.331    GS2u   
1          3.43          3.62     0.988     RES   
2          1.29          0.71     0.430     RES   
3          1.51          2.49     0.810     RES   
4          1.33          0.68     0.398     RES   

                                   uuid  
0  1b3752ec-3a42-4308-bb56-10402d892393  
1  7f87867a-6e14-4922-94d1-b20795832ed4  
2  d96228d6-b650-4ecb-abea-97103c3ca7b1  
3  acbc70e7-8b7d-46b7-aede-af0fd7a66e0a  
4  2b1cdb59-a243-4a13-b42c-c472c260a8e8

In [21]:
df2 = df2[['uuid',gcol]].dropna(how='any') #select only uuid and rce from df2
print(df2.shape)
df2.head()

(198451, 2)


uuid  real_rce
0  1b3752ec-3a42-4308-bb56-10402d892393     7.331
1  7f87867a-6e14-4922-94d1-b20795832ed4     0.988
2  d96228d6-b650-4ecb-abea-97103c3ca7b1     0.430
3  acbc70e7-8b7d-46b7-aede-af0fd7a66e0a     0.810
4  2b1cdb59-a243-4a13-b42c-c472c260a8e8     0.398

In [22]:
df = df.merge(df2,on='uuid') #merge target values to input data
df = df[df[gcol] < 5] #remove rows with very large rce
df = df[df.isnull().sum(axis=1) <= 15] #remove rows with too much missing data

print(df.shape)
df.head()

(169296, 28)


uuid  OccupationRollup  EducationLevel  \
0  86f54722-f331-4140-9c42-2a03b89dd6da                17               3   
1  f6913def-a6b6-4ab0-a74b-1d2ea64369b0                17               5   
2  4fe627d9-31f8-4c8f-be22-3878e910e2b1                17               3   
3  53c1335e-364f-4a56-8923-5aac43603192                17               3   
4  5c3b349d-9c33-4675-ac40-01b08db778b7                17               3   

   longitude  YEAR_BUILT  ESTINCOME  population  state  median_income  \
0   -87.6568        2013          7       66461      2     78622.5332   
1   -87.6568         -99        -99       66461      2     78622.5332   
2   -87.6568        2000          9       66461      2     78622.5332   
3   -87.6568        1899          5       66461      2     78622.5332   
4   -87.6568        1889          9       66461      2     78622.5332   

   Mail Public Responder Indicator  ...  SALE_PRICE  HHSIZE  \
0                                2  ...      950000       3   
1                                2  ...         -99     -99   
2                                2  ...         -99       6   
3                                2  ...     1075000       5   
4                                2  ...      160000       2   

   LAND_SQUARE_FOOTAGE_D  DwellingType  Gender  FuelDescription  \
0                   3125             1       1                9   
1                    -99             0       2                9   
2                   3584             1       1                9   
3                   6240             1       1                9   
4                   3125             1       1                9   

   ASSD_TOTAL_VALUE  median_home_value  TotalRooms  real_rce  
0            147500             344500           0     4.710  
1               -99             344500         -99     2.052  
2            114342             344500           0     2.382  
3            129672             344500           0     4.314  
4             65328             344500           0     0.800  

[5 rows x 28 columns]

In [23]:
du = df[gcol].mean() #find the mean and std of the rce in the input data
ds = df[gcol].std()
din = {'mean': du, 'std': ds}
np.save('scale_rce.npy',din) #save out as a dictionary to file
#return distance from mean in units of std
din

{'mean': 1.0115697535677344, 'std': 0.7575005820623689}

In [24]:
df[gcol] = df[gcol].apply(lambda x: (x - din['mean']) / din['std']) #apply standard scaling to rce column
df.head()

uuid  OccupationRollup  EducationLevel  \
0  86f54722-f331-4140-9c42-2a03b89dd6da                17               3   
1  f6913def-a6b6-4ab0-a74b-1d2ea64369b0                17               5   
2  4fe627d9-31f8-4c8f-be22-3878e910e2b1                17               3   
3  53c1335e-364f-4a56-8923-5aac43603192                17               3   
4  5c3b349d-9c33-4675-ac40-01b08db778b7                17               3   

   longitude  YEAR_BUILT  ESTINCOME  population  state  median_income  \
0   -87.6568        2013          7       66461      2     78622.5332   
1   -87.6568         -99        -99       66461      2     78622.5332   
2   -87.6568        2000          9       66461      2     78622.5332   
3   -87.6568        1899          5       66461      2     78622.5332   
4   -87.6568        1889          9       66461      2     78622.5332   

   Mail Public Responder Indicator  ...  SALE_PRICE  HHSIZE  \
0                                2  ...      950000       3   
1                                2  ...         -99     -99   
2                                2  ...         -99       6   
3                                2  ...     1075000       5   
4                                2  ...      160000       2   

   LAND_SQUARE_FOOTAGE_D  DwellingType  Gender  FuelDescription  \
0                   3125             1       1                9   
1                    -99             0       2                9   
2                   3584             1       1                9   
3                   6240             1       1                9   
4                   3125             1       1                9   

   ASSD_TOTAL_VALUE  median_home_value  TotalRooms  real_rce  
0            147500             344500           0  4.882412  
1               -99             344500         -99  1.373504  
2            114342             344500           0  1.809147  
3            129672             344500           0  4.359641  
4             65328             344500           0 -0.279300  

[5 rows x 28 columns]

In [25]:
df.describe()

OccupationRollup  EducationLevel      longitude     YEAR_BUILT  \
count     169296.000000   169296.000000  169296.000000  169296.000000   
mean          15.089287        3.483431     -79.636058     676.041879   
std            4.035842        1.629854       6.051775     997.443784   
min            0.000000        0.000000    -156.927200     -99.000000   
25%           15.000000        2.000000     -87.701200     -99.000000   
50%           17.000000        4.000000     -77.754000     -99.000000   
75%           17.000000        5.000000     -75.241300    1955.000000   
max           18.000000        5.000000     -68.070400    2018.000000   

           ESTINCOME     population          state  median_income  \
count  169296.000000  169296.000000  169296.000000  169296.000000   
mean      -23.228334   23302.356754       4.406501   59117.705800   
std        46.557943   17163.835523       2.026124   19850.401296   
min       -99.000000       0.000000       0.000000     346.070000   
25%       -99.000000    9932.000000       2.000000   46756.823400   
50%         5.000000   19992.000000       5.000000   55558.818000   
75%         6.000000   31044.000000       7.000000   67201.292700   
max         9.000000  114720.000000       7.000000  223106.175000   

       Mail Public Responder Indicator  Home Value Range  ...    SALE_PRICE  \
count                    169296.000000     169296.000000  ...  1.692960e+05   
mean                          1.193808        -31.145715  ...  4.277395e+04   
std                           0.977329         49.312308  ...  1.608551e+05   
min                           0.000000        -99.000000  ... -9.900000e+01   
25%                           0.000000        -99.000000  ... -9.900000e+01   
50%                           2.000000          2.000000  ... -9.900000e+01   
75%                           2.000000          5.000000  ...  0.000000e+00   
max                           2.000000         15.000000  ...  2.340181e+07   

              HHSIZE  LAND_SQUARE_FOOTAGE_D   DwellingType         Gender  \
count  169296.000000           1.692960e+05  169296.000000  169296.000000   
mean      -23.442391           5.594765e+04       1.501890       1.048318   
std        44.293786           4.611347e+05       0.607607       0.797122   
min       -99.000000          -9.900000e+01       0.000000       0.000000   
25%       -99.000000          -9.900000e+01       1.000000       0.000000   
50%         2.000000          -9.900000e+01       2.000000       1.000000   
75%         3.000000           1.056925e+04       2.000000       2.000000   
max         9.000000           8.651016e+07       2.000000       2.000000   

       FuelDescription  ASSD_TOTAL_VALUE  median_home_value     TotalRooms  \
count    169296.000000      1.692960e+05       1.692960e+05  169296.000000   
mean          8.191523      1.039421e+05       2.529065e+05     -74.433844   
std           2.048039      1.824008e+05       1.746310e+05      43.710354   
min           0.000000     -9.900000e+01       2.340000e+04     -99.000000   
25%           9.000000     -9.900000e+01       1.450000e+05     -99.000000   
50%           9.000000      3.144500e+04       2.093000e+05     -99.000000   
75%           9.000000      1.662960e+05       2.794000e+05     -99.000000   
max          10.000000      2.927565e+07       2.836100e+06     999.000000   

           real_rce  
count  1.692960e+05  
mean  -2.518489e-14  
std    1.000000e+00  
min   -1.335405e+00  
25%   -6.674183e-01  
50%   -2.740193e-01  
75%    3.332410e-01  
max    5.263930e+00  

[8 rows x 27 columns]

In [26]:
df[['uuid']].to_csv("./output_data/X_mapping.csv",index=False) #save the ordering of uuids to file
X = df.drop(['uuid',gcol],1) #save the entire input data set
X.to_csv("./output_data/X.csv",index=False)
y = df[[gcol]] #save the entire target data set
y.to_csv("./output_data/y.csv",index=False)

In [27]:
train, test = train_test_split(df.drop(['uuid'],1), test_size=0.2,random_state=1) #randomly split input data
print(train.shape,test.shape)
train.drop(gcol,1).to_csv("./output_data/train_X.csv",index=False) #save the training data
train[[gcol]].to_csv("./output_data/train_y.csv",index=False)

test.drop(gcol,1).to_csv("./output_data/test_X.csv",index=False) #save the test data
test[[gcol]].to_csv("./output_data/test_y.csv",index=False)

(135436, 27) (33860, 27)
